In [1]:
!pip install qkeras

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 11.7 MB/s eta 0:00:00
  Created wheel for pyparser: filename=pyparser-1.0-py3-none-any.whl size=4914 sha256=0377d8b01f2d67ae1b249c3c4b5820291544400d880e77115832cd60d2a01579
  Stored in directory: /root/.cache/pip/wheels/25/87/78/ff95e8a747dc534fbd199fb3ea06d80935bc87e44567bbdb31
  Created wheel for parse: filename=parse-1.6.5-py3-none-any.whl size=18156 sha256=ef3d35e0d4b68364755648d585b0fa04875f3fda63fcd5a45fbbaca47bc6b77b
  Stored in directory: /root/.cache/pip/wheels/3f/10/c8/5023ea72290855ee33e7bc403e1048ae238b9c2fdb549a9f51
Successfully built pyparser parse


In [2]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_val, y_val)=cifar10.load_data()

print('Shape of x_train is {}'.format(x_train.shape))
print('Shape of x_val is {}'.format(x_val.shape)) 
print('Shape of y_train is {}'.format(y_train.shape))
print('Shape of y_val is {}'.format(y_val.shape))

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Shape of x_train is (50000, 32, 32, 3)
Shape of x_val is (10000, 32, 32, 3)
Shape of y_train is (50000, 1)
Shape of y_val is (10000, 1)


In [3]:
from tensorflow.keras.utils import to_categorical

# Normalizing
x_train=x_train/256.
x_val=x_val/256.

#One hot encoding
y_train_cat=to_categorical(y_train,10)
y_val_cat=to_categorical(y_val,10)

In [4]:
# Importing the necessary libraries, which we may or may not use. Its always good idea to import them befor (if you remember) else we can do it at any point of time no problem.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,Input, AveragePooling2D, Activation,Conv2D, MaxPooling2D, BatchNormalization,Concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [5]:
model1=Sequential()
model1.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Activation("relu"))
model1.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model1.add(BatchNormalization())
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Activation("relu"))
model1.add(Flatten())
model1.add(Dense(64,activation='relu'))
model1.add(Dense(10,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 29, 29, 32)          │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 29, 29, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 32)          │          16,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 11, 11, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 70,154 (274.04 KB)

 Trainable params: 70,026 (273.54 KB)

 Non-trainable params: 128 (512.00 B)

In [7]:
history1=model1.fit(x_train,
                    y_train_cat,
                    epochs=100,
                    validation_data=(x_val,y_val_cat)
                   )

Epoch 1/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 50s 30ms/step - accuracy: 0.3982 - loss: 1.6891 - val_accuracy: 0.4087 - val_loss: 1.9179
Epoch 2/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 47s 30ms/step - accuracy: 0.5948 - loss: 1.1507 - val_accuracy: 0.5169 - val_loss: 1.3667
Epoch 3/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.6535 - loss: 0.9872 - val_accuracy: 0.5270 - val_loss: 1.4440
Epoch 4/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 29ms/step - accuracy: 0.6942 - loss: 0.8807 - val_accuracy: 0.6200 - val_loss: 1.1050
Epoch 5/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.7193 - loss: 0.8017 - val_accuracy: 0.6645 - val_loss: 0.9783
Epoch 6/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.7363 - loss: 0.7516 - val_accuracy: 0.4832 - val_loss: 1.8079
Epoch 7/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.7539 - loss: 0.6972 - val_accuracy: 0.5702 - val_loss: 1.4969
Epoch 8/100
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 47s 30ms/step - accuracy: 

In [8]:
import pandas as pd
import numpy as np

# Load test data
test_data = np.load("/kaggle/input/hls-4-ml-mlperf-tiny-image-classification/test_data.npy")
print(test_data.shape)

(10000, 32, 32, 3)


In [9]:
test_data = test_data/256.

In [10]:


test_predict = model1.predict(test_data)
test_predict_inx = np.argmax(test_predict, axis=-1)


313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step


In [11]:
test_predict_inx

array([1, 6, 8, ..., 8, 7, 4])

In [12]:
num_samples = len(test_predict_inx)

# Create a DataFrame
submission_df = pd.DataFrame({
    "id": np.arange(num_samples),
    "label": test_predict_inx  # Prediction Class Index
})


In [13]:
# Save the submission file
submission_csv_path = "submission_v1.csv"
submission_df.to_csv(submission_csv_path, index=False)

print(f"Submission file saved: {submission_csv_path}")

Submission file saved: submission_v1.csv
